# Case-Study: Telecom Churn

* Usage-based churn determination
* Churn to be predicted for September based on June, July and August data
* Phase coding:
    * Good Month: June, July
    * Action Month: Aug
    * Churn Month: Sept


* Churn needs to be predicted for High-Value customers; How to find these customers?
    * Recharge Amount > X, where X is the 70th percentile of Recharge Amount distribution


* How to perform Churn Tagging?
    * Churn tagging will be done based on information of the 4th month (Sept)
    * Condition 1: No calls made in that month
    * Condition 2: No mobile internet usage in that month
    * Condition 1 AND Condition 2
    * Attributes indicated in the Problem Statement: total_ic_mou_9, total_og_mou_9, vol_2g_mb_9, vol_3g_mb_9
    * IMP: After churner tagging, the features of September are no longer required for the analysis and model building, and we can remove these features


* Main goals of the model/models:
    * Predict which customer will churn
    * Identify the important features to consider for the analysis


* Recommended approach: PCA --> Classifier
* Class imbalance is to be expected


* Suggested steps to build the model [ONLY PREDICTION PART] (for reference):
    * Preprocess data (convert columns to appropriate formats, handle missing values, etc.)
    * Conduct appropriate exploratory analysis to extract useful insights (whether directly useful for business or for eventual modelling/feature engineering).
    * Derive new features.
    * Reduce the number of variables using PCA.
    * Train a variety of models, tune model hyperparameters, etc. (handle class imbalance using appropriate techniques).
    * Evaluate the models using appropriate evaluation metrics. Note that is is more important to identify churners than the non-churners accurately - choose an appropriate evaluation metric which reflects this business goal.
    * Finally, choose a model based on some evaluation metric.
    

* Use of LR or Decision Trees to identify useful features for the sake of the business user; Highlights of the feature usefulness with the help of plots

* Closing: Interpretation of the results by suggestion of strategies

In [1]:
import os
dirname = os.getcwd()
print('Path: ', dirname)
print('\nFiles in OS Path:\n',os.listdir(dirname))

Path:  G:\UpGrad\03_ML_2\07_CaseStudy_TelecomChurn

Files in OS Path:
 ['.ipynb_checkpoints', 'Data+Dictionary-+Telecom+Churn+Case+Study.xlsx', 'telecom_churn_data.csv', 'Untitled.ipynb', '~$Data+Dictionary-+Telecom+Churn+Case+Study.xlsx']


In [2]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
dataframe = pd.read_csv('telecom_churn_data.csv')
dataframe.shape

(99999, 226)

In [4]:
dataframe.head()

,mobile_number,circle_id,loc_og_t2o_mou,std_og_t2o_mou,loc_ic_t2o_mou,last_date_of_month_6,last_date_of_month_7,last_date_of_month_8,last_date_of_month_9,arpu_6,...,sachet_3g_9,fb_user_6,fb_user_7,fb_user_8,fb_user_9,aon,aug_vbc_3g,jul_vbc_3g,jun_vbc_3g,sep_vbc_3g
0,7000842753,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,197.385,...,0,1.0,1.0,1.0,NaN,968,30.4,0.0,101.20,3.58
1,7001865778,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,34.047,...,0,NaN,1.0,1.0,NaN,1006,0.0,0.0,0.00,0.00
2,7001625959,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,167.690,...,0,NaN,NaN,NaN,1.0,1103,0.0,0.0,4.17,0.00
3,7001204172,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,221.338,...,0,NaN,NaN,NaN,NaN,2491,0.0,0.0,0.00,0.00
4,7000142493,109,0.0,0.0,0.0,6/30/2014,7/31/2014,8/31/2014,9/30/2014,261.636,...,0,0.0,NaN,NaN,NaN,1526,0.0,0.0,0.00,0.00


**Filtering for the High-Value Customers**

* Let us consolidate the total recharge amount (total_rech_amt_x) for the various months, as this is the most likely derived feature based on which the filtering is to be performed.
* We need to obtain the Average Recharge Amount for the Good Months

In [5]:
# Filter by the 70th Percentile of the Recharge Amount
rech_cols = [col for col in dataframe.columns if 'rech' in col]
print(rech_cols)

# Determining High-Value Customers based on consolidated recharge amounts of the 'Good' months
dataframe['rech_aggregate_6_7'] = (dataframe['total_rech_amt_6'] + dataframe['total_rech_amt_7'])/2
print('\n\nAggregate Recharge\n', dataframe['rech_aggregate_6_7'].head())

# 70th percentile Amount
rech_70 = dataframe['rech_aggregate_6_7'].quantile(0.7)

# Filter
dataframe = dataframe[dataframe['rech_aggregate_6_7'] >= rech_70]
print('\n\nShape after filtering: ',dataframe.shape)

['total_rech_num_6', 'total_rech_num_7', 'total_rech_num_8', 'total_rech_num_9', 'total_rech_amt_6', 'total_rech_amt_7', 'total_rech_amt_8', 'total_rech_amt_9', 'max_rech_amt_6', 'max_rech_amt_7', 'max_rech_amt_8', 'max_rech_amt_9', 'date_of_last_rech_6', 'date_of_last_rech_7', 'date_of_last_rech_8', 'date_of_last_rech_9', 'date_of_last_rech_data_6', 'date_of_last_rech_data_7', 'date_of_last_rech_data_8', 'date_of_last_rech_data_9', 'total_rech_data_6', 'total_rech_data_7', 'total_rech_data_8', 'total_rech_data_9', 'max_rech_data_6', 'max_rech_data_7', 'max_rech_data_8', 'max_rech_data_9', 'count_rech_2g_6', 'count_rech_2g_7', 'count_rech_2g_8', 'count_rech_2g_9', 'count_rech_3g_6', 'count_rech_3g_7', 'count_rech_3g_8', 'count_rech_3g_9', 'av_rech_amt_data_6', 'av_rech_amt_data_7', 'av_rech_amt_data_8', 'av_rech_amt_data_9']


Aggregate Recharge
 0    307.0
1    229.0
2    241.5
3    270.0
4    273.0
Name: rech_aggregate_6_7, dtype: float64


Shape after filtering:  (30011, 227)


In [6]:
# Checking if there are missing values in this column, though we expect there to be none
dataframe['rech_aggregate_6_7'].isna().sum()

0

* A lot of the features are given specifically per month (6-9 for June to Sept); Let us check which columns are not specific to any month
* Not specific to month: mobile_number, circle_id, 

In [7]:
# 0 calls AND 0 MB data --> Churn
dataframe[['total_ic_mou_9', 'total_og_mou_9', 'vol_2g_mb_9', 'vol_3g_mb_9']]

dataframe['Churn'] = np.nan
for i in list(dataframe.index):
    cond1 = round(dataframe['total_ic_mou_9'][i] + dataframe['total_og_mou_9'][i])
    cond2 = round(dataframe['vol_2g_mb_9'][i] + dataframe['vol_3g_mb_9'][i])
    if (cond1 == 0) & (cond2 == 0):
        dataframe['Churn'][i] = 1
    else:
        dataframe['Churn'][i] = 0

dataframe['Churn']

I:\ML_Anaconda_Python_3.6\Anaconda\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  if __name__ == '__main__':
I:\ML_Anaconda_Python_3.6\Anaconda\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()


7        1.0
8        0.0
13       0.0
16       0.0
17       0.0
        ... 
99970    0.0
99974    0.0
99986    0.0
99988    0.0
99997    0.0
Name: Churn, Length: 30011, dtype: float64

In [8]:
dataframe.shape

(30011, 228)

In [9]:
# Since we have completed Churn Tagging, 
# the features of September are no longer required for the analysis 
# and model building, so we can remove these features

data_sept = [col for col in dataframe.columns if '_9' in col]

dataframe = dataframe.drop(data_sept, axis = 1)
dataframe.shape, dataframe.columns

((30011, 174),
 Index(['mobile_number', 'circle_id', 'loc_og_t2o_mou', 'std_og_t2o_mou',
        'loc_ic_t2o_mou', 'last_date_of_month_6', 'last_date_of_month_7',
        'last_date_of_month_8', 'arpu_6', 'arpu_7',
        ...
        'fb_user_6', 'fb_user_7', 'fb_user_8', 'aon', 'aug_vbc_3g',
        'jul_vbc_3g', 'jun_vbc_3g', 'sep_vbc_3g', 'rech_aggregate_6_7',
        'Churn'],
       dtype='object', length=174))

**===============================================================================**

**Missing Value Analysis**

* Drop List 1 will consist of all features that have more than 50% missing values

In [12]:
# Get the list of columns that have more than 50% missing values

drop_list_50 = []
for col in dataframe.columns:
    if (dataframe[col].isna().sum()/dataframe.shape[0]) >= 0.5:
        print(col,'\t',100*dataframe[col].isna().sum()/dataframe.shape[0])
        drop_list_50.append(col)

date_of_last_rech_data_6 	 62.0239245609943
date_of_last_rech_data_7 	 61.14091499783413
date_of_last_rech_data_8 	 60.834360734397386
total_rech_data_6 	 62.0239245609943
total_rech_data_7 	 61.14091499783413
total_rech_data_8 	 60.834360734397386
max_rech_data_6 	 62.0239245609943
max_rech_data_7 	 61.14091499783413
max_rech_data_8 	 60.834360734397386
count_rech_2g_6 	 62.0239245609943
count_rech_2g_7 	 61.14091499783413
count_rech_2g_8 	 60.834360734397386
count_rech_3g_6 	 62.0239245609943
count_rech_3g_7 	 61.14091499783413
count_rech_3g_8 	 60.834360734397386
av_rech_amt_data_6 	 62.0239245609943
av_rech_amt_data_7 	 61.14091499783413
av_rech_amt_data_8 	 60.834360734397386
arpu_3g_6 	 62.0239245609943
arpu_3g_7 	 61.14091499783413
arpu_3g_8 	 60.834360734397386
arpu_2g_6 	 62.0239245609943
arpu_2g_7 	 61.14091499783413
arpu_2g_8 	 60.834360734397386
night_pck_user_6 	 62.0239245609943
night_pck_user_7 	 61.14091499783413
night_pck_user_8 	 60.834360734397386
fb_user_6 	 62.0239

In [ ]:
'''# Dropping the columns that have excessive missing values
dataframe = dataframe.drop(drop_list_50, axis = 1)
print('\n\nNew Shape: ', dataframe.shape)'''

In [11]:
# Printing all the remaining columns, grouped by month
print('Features: ', len([col for col in dataframe.columns if '_6' in col]), 
      '\t',*[col for col in dataframe.columns if '_6' in col], sep = "|:|     |:|")
print('\n\n','Features: ', len([col for col in dataframe.columns if '_7' in col]), 
      '\t',*[col for col in dataframe.columns if '_7' in col], sep = "|:|     |:|")
print('\n\n','Features: ', len([col for col in dataframe.columns if '_8' in col]), 
      '\t',*[col for col in dataframe.columns if '_8' in col], sep = "|:|     |:|")
print('\n\n',*[col for col in dataframe.columns if ('_6' not in col) & ('_7' not in col) & ('_8' not in col)], 
      sep = "|:|     |:|")

Features: |:|     |:|55|:|     |:|	|:|     |:|last_date_of_month_6|:|     |:|arpu_6|:|     |:|onnet_mou_6|:|     |:|offnet_mou_6|:|     |:|roam_ic_mou_6|:|     |:|roam_og_mou_6|:|     |:|loc_og_t2t_mou_6|:|     |:|loc_og_t2m_mou_6|:|     |:|loc_og_t2f_mou_6|:|     |:|loc_og_t2c_mou_6|:|     |:|loc_og_mou_6|:|     |:|std_og_t2t_mou_6|:|     |:|std_og_t2m_mou_6|:|     |:|std_og_t2f_mou_6|:|     |:|std_og_t2c_mou_6|:|     |:|std_og_mou_6|:|     |:|isd_og_mou_6|:|     |:|spl_og_mou_6|:|     |:|og_others_6|:|     |:|total_og_mou_6|:|     |:|loc_ic_t2t_mou_6|:|     |:|loc_ic_t2m_mou_6|:|     |:|loc_ic_t2f_mou_6|:|     |:|loc_ic_mou_6|:|     |:|std_ic_t2t_mou_6|:|     |:|std_ic_t2m_mou_6|:|     |:|std_ic_t2f_mou_6|:|     |:|std_ic_t2o_mou_6|:|     |:|std_ic_mou_6|:|     |:|total_ic_mou_6|:|     |:|spl_ic_mou_6|:|     |:|isd_ic_mou_6|:|     |:|ic_others_6|:|     |:|total_rech_num_6|:|     |:|total_rech_amt_6|:|     |:|max_rech_amt_6|:|     |:|date_of_last_rech_6|:|     |:|last_day_rch_amt_6|:|

The Significance of each type of feature (referring the Data Dictionary):

* Month-wise features:
    * last_date_of_month_X: Probably the last date of telecom activity for that month
    * arpu_X: Average Revenue per User (Thiss looks like a significant feature for the company)
    
    * onnet_mou_X: Onnet deals with all calls made within the operator network (same); mou  is the minutes of usage of the call service; Therefore, the feature would provide us data on the total intra-operator call duration for the specified month
    * offnet_mou_X: Total inter-operator (accross operators) call duration for the specified month
    * roam_ic_mou_X: Total duration of roaming incoming calls for the specified month
    * roam_og_mou_X: Same for outgoing calls
    
    * loc_og_t2t_mou_X: T2T implies intra-operator, so this may be a subset of onnet_mou_X; We need to check if this is the case or not
    * loc_og_t2m_mou_X: T2M is for inter-operator calls (**subset of offnet_mou_X?**)
    * loc_og_t2f_mou_X: T2F is an intra-operator case where fixed-lines (landlines) are involved
    * loc_of_t2c_mou_X: T2C calls are towards the call centre, so it is possible that these types of calls do not generate revenue (**needs to be checked...**)
    * loc_og_mou_X: This can be expected to be a super-set of all loc_og's (**to be tested**)
    * std_og_t2t_mou_X: the T2T, T2M, T2F and T2C are all related to STD calls for these types of features (incl std_og's)
    * isd_og_mou_X: isd_og has only the total duration feature (even before handling of missing values)
    * spl_og_mou_X: Special Calls
    * og_others_X: Self-explanatory
    * total_og_mou_X: Superset of all OG's for that month (5,5,1,1,1)
    
    * Incoming Call durations for local, std, isd, special and others (and T2T, T2M, T2F for local and STD) (T2C undetected) (STD has additional T2O, for landlines from other operators); also, totals (4,5,1,1,1)
    
    * total_rech_num_X: Probably the number of times a recharge was done within the specified month
    * total_rech_amt_X: Total recharge-related payment made by the customer in the specified month
    * max_rech_amt_X: Largest single recharge denomination
    * date_of_last_rech_X: Self-explanatory
    * last_day_rch_amt_X: Related
    
    * date_of_last_rech_data_X: 
    * total_rech_data_X: 
    * max_rech_data_X: 
    * av_rech_amt_data_X: 
    
    * night_pck_user_X: (PACKS - prepaid schemes; for specific hours of the night)
    
    * count_rech_2g_X, vol_2g_mb_X, arpu_2g_X, monthly_2g_X, 
    
    * count_rech_3g_X, vol_3g_mb_X, arpu_3g_X, monthly_3g_X, 
* Generic Features:
    * mobile_number
    * circle_id
    * loc_og_t2o_mou
    * std_og_t2o_mou
    * loc_ic_t2o_mou
    * aon: Number of days the specific user has used the network
    * aug_vbc_3g, jun_vbc_3g, jul_vbc_3g, sep_vbc_3g: Volume based cost (for non-specified plan); The September feature needs to be removed